In [2]:
while True:
    try:
        x = int(input("Please enter a number: "))
        break
    except ValueError:
        print("Oops!  That was no valid number.  Try again...")

Oops!  That was no valid number.  Try again...
Oops!  That was no valid number.  Try again...
Oops!  That was no valid number.  Try again...
Oops!  That was no valid number.  Try again...
Oops!  That was no valid number.  Try again...
Oops!  That was no valid number.  Try again...
Oops!  That was no valid number.  Try again...
Oops!  That was no valid number.  Try again...
Oops!  That was no valid number.  Try again...
Oops!  That was no valid number.  Try again...


A class in an `except` clause is compatible with an exception if it is the same class or a base class thereof (but not the other way around — an except clause listing a derived class is not compatible with a base class). For example, the following code will print B, C, D in that order:


Note that if the *except clauses* were reversed (with `except B` first), it would have printed B, B, B — the first matching *except clause* is triggered. It checks the dymanic class: Is `C` a `B`? In this example, put base class at the end to trigger the right handler.

In [6]:
class B(Exception):  # derived class
    pass

class C(B):
    pass

class D(C):
    pass

for cls in [B, C, D]:
    try:
        raise cls()
    except D:
        print("D")
    except C:
        print("C")
    except B:
        print("B")
    

B
C
D


In [7]:
try:
    raise Exception('spam', 'eggs')  # args list, Exception is builtin
except Exception as inst:  # alias for Exception
    print(type(inst))    # the exception type
    print(inst.args)     # arguments stored in .args
    print(inst)          # __str__ allows args to be printed directly,
                         # but may be overridden in exception subclasses
    x, y = inst.args     # unpack args
    print('x =', x)
    print('y =', y)

<class 'Exception'>
('spam', 'eggs')
('spam', 'eggs')
x = spam
y = eggs


One of its subclasses, [`Exception`](https://docs.python.org/3/library/exceptions.html#Exception), is the base class of all the non-fatal exceptions.

The most common pattern for handling [`Exception`](https://docs.python.org/3/library/exceptions.html#Exception) is to print or log the exception and then re-raise it (allowing a caller to handle the exception as well):

In [13]:
import sys

try:
    f = open('myfile.txt')
    s = f.readline()
    i = int(s.strip())
    print(f"i = {i}")
except OSError:
    print("OS error:", OSError)
except ValueError:
    print("Could not convert data to an integer.")
except Exception as err:
    print(f"Unexpected {err=}, {type(err)=}")
    raise

Could not convert data to an integer.


In [ ]:
for arg in sys.argv[1:]:
    try:
        f = open(arg, 'r')
    except OSError:
        print('cannot open', arg)
    else:
        print(arg, 'has', len(f.readlines()), 'lines')
        f.close()

The [`try`](https://docs.python.org/3/reference/compound_stmts.html#try) … [`except`](https://docs.python.org/3/reference/compound_stmts.html#except) statement has an optional *else clause*, which, when present, must follow all *except clauses*. It is useful for code that must be executed if the *try clause* does not raise an exception. For example:

The use of the `else` clause is better than adding additional code to the [`try`](https://docs.python.org/3/reference/compound_stmts.html#try) clause because it avoids accidentally catching an exception that wasn’t raised by the code being protected by the `try` … `except` statement.

Exception handlers do not handle only exceptions that occur immediately in the try clause, but also those that occur inside functions that are called (even indirectly) in the try clause. For example:

In [14]:
def this_fails():
    x = 1/0

try:
    this_fails()
except ZeroDivisionError as err:
    print('Handling run-time error:', err)

Handling run-time error: division by zero


In [17]:
try:
    raise NameError('HiThere')
except NameError:
    print('An exception flew by!')
    raise  # re-raise, a shorthand

An exception flew by!


NameError: HiThere

In [18]:
def divide(x, y):
    try:
        result = x / y
    except ZeroDivisionError:
        print("division by zero!")
    else:
        print("result is", result)
    finally:
        print("executing finally clause")


In [19]:
divide(2, 1)

result is 2.0
executing finally clause


In [20]:
divide(2,0)

division by zero!
executing finally clause


In [22]:
divide("2", "1")  # exc is re-raised

executing finally clause


TypeError: unsupported operand type(s) for /: 'str' and 'str'

In real world applications, the [`finally`](https://docs.python.org/3/reference/compound_stmts.html#finally) clause is useful for releasing external resources (such as files or network connections), regardless of whether the use of the resource was successful.

In [23]:
with open("myfile.txt") as f:
    for line in f:
        print(line, end="")

   Aha, you find me! Smart boy!   
   26
   Aha, you find me! Smart boy!   

There are situations where it is necessary to report several exceptions that have occurred. This is often the case in concurrency frameworks, when several tasks may have failed in parallel, but there are also other use cases where it is desirable to continue execution and collect multiple errors rather than raise the first exception.

The builtin [`ExceptionGroup`](https://docs.python.org/3/library/exceptions.html#ExceptionGroup) wraps a list of exception instances so that they can be raised together. It is an exception itself, so it can be caught like any other exception.

In [1]:
def f():
    excs = [OSError('error 1'), SystemError('error 2')]
    raise ExceptionGroup('there were problems', excs)

f()

ExceptionGroup: there were problems (2 sub-exceptions)

By using `except*` instead of `except`, we can selectively handle only the exceptions in the group that match a certain type. In the following example, which shows a nested exception group, each `except*` clause extracts from the group exceptions of a certain type while letting all other exceptions propagate to other clauses and eventually to be reraised.

In [1]:
def f():
    raise ExceptionGroup(
        "group1",
        [
            OSError(1),
            SystemError(2),
            ExceptionGroup(
                "group2",
                [
                    OSError(3),
                    RecursionError(4)
                ]
            )
        ]
    )

try:
    f()
except* OSError as e:
    print("There were OSErrors")
except* SystemError as e:
    print("There were SystemErrors")

There were OSErrors
There were SystemErrors


ExceptionGroup: group1 (1 sub-exception)

In [11]:
try:
    raise TypeError('bad type')
except TypeError as e:
    e.add_note('Add some information')
    e.add_note('Add some more information')
    raise
    

TypeError: bad type

In [13]:
def f():
    raise OSError('operation failed')

excs = []
for i in range(3):
    try:
        f()
    except Exception as e:
        e.add_note(f'Happened in Iteration {i+1}')
        excs.append(e)

raise ExceptionGroup('We have some problems', excs)

# The complete result is shown in console, not in notebook.

ExceptionGroup: We have some problems (3 sub-exceptions)